# Bulgarian Personal Numbers

## Introduction

The function `clean_bg_pnf()` cleans a column containing Bulgarian personal number of a foreigner, and standardizes them in a given format. The function `validate_bg_pnf()` validates either a single PNF strings, a column of PNF strings or a DataFrame of PNF strings, returning `True` if the value is valid, and `False` otherwise.

PNF strings can be converted to the following formats via the `output_format` parameter:

* `compact`: only number strings without any seperators or whitespace, like "7111042925"
* `standard`: PNF strings with proper whitespace in the proper places. Note that in the case of PNF, the compact format is the same as the standard one.

Invalid parsing is handled with the `errors` parameter:

* `coerce` (default): invalid parsing will be set to NaN
* `ignore`: invalid parsing will return the input
* `raise`: invalid parsing will raise an exception

The following sections demonstrate the functionality of `clean_bg_pnf()` and `validate_bg_pnf()`. 

### An example dataset containing PNF strings

In [1]:
import pandas as pd
import numpy as np
df = pd.DataFrame(
    {
        "pnf": [
            '7111 042 925',
            '7111042922',  # invalid check digit
            '71110A2922',  # invalid digit
            "002 724 334", # not PNF
            "hello",
            np.nan,
            "NULL",
        ], 
        "address": [
            "123 Pine Ave.",
            "main st",
            "1234 west main heights 57033",
            "apt 1 789 s maple rd manhattan",
            "1111 S Figueroa St, Los Angeles, CA 90015",
            "(staples center) 1111 S Figueroa St, Los Angeles",
            "hello",
        ]
    }
)
df

,pnf,address
0,7111 042 925,123 Pine Ave.
1,7111042922,main st
2,71110A2922,1234 west main heights 57033
3,002 724 334,apt 1 789 s maple rd manhattan
4,hello,"1111 S Figueroa St, Los Angeles, CA 90015"
5,NaN,"(staples center) 1111 S Figueroa St, Los Angeles"
6,NULL,hello


## 1. Default `clean_bg_pnf`

By default, `clean_bg_pnf` will clean pnf strings and output them in the standard format with proper separators.

In [2]:
from dataprep.clean import clean_bg_pnf
clean_bg_pnf(df, column = "pnf")

,pnf,address,pnf_clean
0,7111 042 925,123 Pine Ave.,7111042925
1,7111042922,main st,NaN
2,71110A2922,1234 west main heights 57033,NaN
3,002 724 334,apt 1 789 s maple rd manhattan,NaN
4,hello,"1111 S Figueroa St, Los Angeles, CA 90015",NaN
5,NaN,"(staples center) 1111 S Figueroa St, Los Angeles",NaN
6,NULL,hello,NaN


## 2. Output formats

This section demonstrates the output parameter.

### `standard` (default)

In [3]:
clean_bg_pnf(df, column = "pnf", output_format="standard")

,pnf,address,pnf_clean
0,7111 042 925,123 Pine Ave.,7111042925
1,7111042922,main st,NaN
2,71110A2922,1234 west main heights 57033,NaN
3,002 724 334,apt 1 789 s maple rd manhattan,NaN
4,hello,"1111 S Figueroa St, Los Angeles, CA 90015",NaN
5,NaN,"(staples center) 1111 S Figueroa St, Los Angeles",NaN
6,NULL,hello,NaN


### `compact`

In [4]:
clean_bg_pnf(df, column = "pnf", output_format="compact")

,pnf,address,pnf_clean
0,7111 042 925,123 Pine Ave.,7111042925
1,7111042922,main st,NaN
2,71110A2922,1234 west main heights 57033,NaN
3,002 724 334,apt 1 789 s maple rd manhattan,NaN
4,hello,"1111 S Figueroa St, Los Angeles, CA 90015",NaN
5,NaN,"(staples center) 1111 S Figueroa St, Los Angeles",NaN
6,NULL,hello,NaN


## 3. `inplace` parameter

This deletes the given column from the returned DataFrame. 
A new column containing cleaned PNF strings is added with a title in the format `"{original title}_clean"`.

In [5]:
clean_bg_pnf(df, column="pnf", inplace=True)

,pnf_clean,address
0,7111042925,123 Pine Ave.
1,NaN,main st
2,NaN,1234 west main heights 57033
3,NaN,apt 1 789 s maple rd manhattan
4,NaN,"1111 S Figueroa St, Los Angeles, CA 90015"
5,NaN,"(staples center) 1111 S Figueroa St, Los Angeles"
6,NaN,hello


## 4. `errors` parameter

### `coerce` (default)

In [6]:
clean_bg_pnf(df, "pnf", errors="coerce")

,pnf,address,pnf_clean
0,7111 042 925,123 Pine Ave.,7111042925
1,7111042922,main st,NaN
2,71110A2922,1234 west main heights 57033,NaN
3,002 724 334,apt 1 789 s maple rd manhattan,NaN
4,hello,"1111 S Figueroa St, Los Angeles, CA 90015",NaN
5,NaN,"(staples center) 1111 S Figueroa St, Los Angeles",NaN
6,NULL,hello,NaN


### `ignore`

In [7]:
clean_bg_pnf(df, "pnf", errors="ignore")

,pnf,address,pnf_clean
0,7111 042 925,123 Pine Ave.,7111042925
1,7111042922,main st,7111042922
2,71110A2922,1234 west main heights 57033,71110A2922
3,002 724 334,apt 1 789 s maple rd manhattan,002 724 334
4,hello,"1111 S Figueroa St, Los Angeles, CA 90015",hello
5,NaN,"(staples center) 1111 S Figueroa St, Los Angeles",NaN
6,NULL,hello,NaN


## 4. `validate_bg_pnf()`

`validate_bg_pnf()` returns `True` when the input is a valid PNF. Otherwise it returns `False`.

The input of `validate_bg_pnf()` can be a string, a Pandas DataSeries, a Dask DataSeries, a Pandas DataFrame and a dask DataFrame.

When the input is a string, a Pandas DataSeries or a Dask DataSeries, user doesn't need to specify a column name to be validated. 

When the input is a Pandas DataFrame or a dask DataFrame, user can both specify or not specify a column name to be validated. If user specify the column name, `validate_bg_pnf()` only returns the validation result for the specified column. If user doesn't specify the column name, `validate_bg_pnf()` returns the validation result for the whole DataFrame.

In [8]:
from dataprep.clean import validate_bg_pnf
print(validate_bg_pnf('7111 042 925'))
print(validate_bg_pnf('7111042922'))
print(validate_bg_pnf('71110A2922' ))
print(validate_bg_pnf('BE431150351'))
print(validate_bg_pnf("004085616"))
print(validate_bg_pnf("hello"))
print(validate_bg_pnf(np.nan))
print(validate_bg_pnf("NULL"))

True
False
False
False
False
False
False
False


### Series

In [9]:
validate_bg_pnf(df["pnf"])

0     True
1    False
2    False
3    False
4    False
5    False
6    False
Name: pnf, dtype: bool

### DataFrame + Specify Column

In [10]:
validate_bg_pnf(df, column="pnf")

0     True
1    False
2    False
3    False
4    False
5    False
6    False
Name: pnf, dtype: bool

### Only DataFrame

In [11]:
validate_bg_pnf(df)

,pnf,address
0,True,False
1,False,False
2,False,False
3,False,False
4,False,False
5,False,False
6,False,False
